In [1]:
import pandas as pd
import numpy as np
import os
import csv

In [2]:
def concatenate_names(name, next_part):
    return f'{name} {next_part.strip()}' if not any(char.isdigit() for char in next_part) else name

def parser(filename, output_file_path, e_type = "gov"):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        new_data = []
        headers = next(reader)
        if e_type == "gov":
            while True:
                try:
                    row = next(reader)
                    new_row = row[:10]  # Copy the first 10 elements as they are
                    if row[11] and any(char.isdigit() for char in row[11]):
                        datarow = row[11].strip()
                    elif row[12] and any(char.isdigit() for char in row[12]):
                        datarow = row[12].strip()    
                    # Concatenate the first nominee name
                    dem_nominee = row[10].strip()
                    if row[11] and not any(char.isdigit() for char in row[11]):
                        dem_nominee = row[11].strip()+" "+dem_nominee

                    # Concatenate the second nominee name only if it doesn't contain digits
                    if row[11] and any(char.isdigit() for char in row[11]):
                        if row[13] and not any(char.isdigit() for char in row[13]):
                            rep_nominee = row[13].strip()+" "+row[12].strip()
                            ex = row[14:]
                        else:
                            rep_nominee = row[12].strip()
                            ex = row[13:]
                    elif row[12] and any(char.isdigit() for char in row[12]):
                        if row[14] and not any(char.isdigit() for char in row[14]):
                            rep_nominee = row[14].strip()+" "+row[13].strip()
                            ex = row[15:]
                        else:
                            rep_nominee = row[13]
                            ex = row[14:]
                    new_row.extend([dem_nominee, datarow, rep_nominee] + ex)
                    new_data.append(new_row)

                except StopIteration:
                    break
        if e_type == "sen":
            while True:
                try:
                    row = next(reader)
                    new_row = row[:11]  # Copy the first 10 elements as they are
                    if row[12] and any(char.isdigit() for char in row[12]):
                        datarow = row[12].strip()
                    elif row[13] and any(char.isdigit() for char in row[13]):
                        datarow = row[13].strip()    
                    # Concatenate the first nominee name
                    dem_nominee = row[11].strip()
                    if row[12] and not any(char.isdigit() for char in row[12]):
                        dem_nominee = row[12].strip()+" "+dem_nominee

                    # Concatenate the second nominee name only if it doesn't contain digits
                    if row[12] and any(char.isdigit() for char in row[12]):
                        if row[14] and not any(char.isdigit() for char in row[14]):
                            rep_nominee = row[14].strip()+" "+row[13].strip()
                            ex = row[15:]
                        else:
                            rep_nominee = row[13].strip()
                            ex = row[14:]
                    elif row[13] and any(char.isdigit() for char in row[13]):
                        if row[14] and not any(char.isdigit() for char in row[14]):
                            rep_nominee = row[15].strip()+" "+row[14].strip()
                            ex = row[16:]
                        else:
                            rep_nominee = row[14]
                            ex = row[15:]
                    new_row.extend([dem_nominee, datarow, rep_nominee] + ex)
                    new_data.append(new_row)

                except StopIteration:
                    break

    # Write to CSV


    with open(output_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        csv_writer.writerows(new_data)

In [3]:
presdata = pd.read_csv("data/1868-2020pres")
    

In [4]:
parser("data/1868-2020gov", "data/parsed1868-2020gov")
govdata = pd.read_csv("data/parsed1868-2020gov")


C:\Users\toomuchdust\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3266: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
parser("data/1868-2020sen", "data/parsed1868-2020sen", "sen")
sendata = pd.read_csv("data/parsed1868-2020sen")

C:\Users\toomuchdust\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3266: DtypeWarning: Columns (17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
sendata = sendata.drop(['election_id',"seat_class","county_first_date","county_end_date","state_admission_date","original_county_name","original_name_end_date"], axis=1)
sendata.rename(columns={'seat_status': 'sen_seat_status', 'democratic_raw_votes': 'sen_democratic_raw_votes', 'dem_nominee': 'sen_dem_nominee', 'republican_raw_votes':'sen_republican_raw_votes',"rep_nominee":"sen_rep_nominee",'republican_raw_votes':'sen_republican_raw_votes', 'raw_county_vote_totals':'sen_raw_county_vote_totals'}, inplace=True)
sendata

,election_year,fips,county_name,state,sfips,office,election_type,sen_seat_status,sen_democratic_raw_votes,sen_dem_nominee,sen_republican_raw_votes,sen_rep_nominee,senate_raw_county_vote_totals_two_party,sen_raw_county_vote_totals
0,1914,1001,AUTAUGA,AL,1,SEN,G,Democratic open seat,525.0,Oscar W. Underwood,15,Alex C. Birch,540.0,602.0
1,1918,1001,AUTAUGA,AL,1,SEN,G,Democratic Senator re-election,507.0,John H. Bankhead,0,None,507.0,507.0
2,1920,1001,AUTAUGA,AL,1,SEN,G,Democratic Senator re-election,957.0,Oscar W. Underwood,167,L.H. Reynolds,1124.0,1130.0
3,1920,1001,AUTAUGA,AL,1,SEN,S,Democratic open seat,1014.0,J. Thomas Heflin,143,C.P. Lunsford,1157.0,1164.0
4,1924,1001,AUTAUGA,AL,1,SEN,G,Democratic Senator re-election,846.0,J. Thomas Heflin,101,F.H. Lathrop,947.0,947.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118682,2008,56045,WESTON,WY,56,SEN,S,Appointed Republican Senator Election,572.0,Nick Carter,2789,John Barrasso,3361.0,3363.0
118683,2012,56045,WESTON,WY,56,SEN,G,Republican Senator re-election,336.0,Tim Chesnut,2887,John Barrasso,3223.0,3346.0
118684,2014,56045,WESTON,WY,56,SEN,G,Republican Senator re-election,194.0,Charlie Hardy,2027,Mike Enzi,2221.0,2485.0
118685,2018,56045,WESTON,WY,56,SEN,G,Republican Senator re-election,321.0,Gary Trauner,2275,John Barrasso,2596.0,2688.0


In [7]:
govdata = govdata.drop(['election_id',"county_first_date","county_end_date","state_admission_date","original_county_name","original_name_end_date", "temp"], axis=1)
govdata.rename(columns={'seat_status': 'gov_seat_status', 'democratic_raw_votes': 'gov_democratic_raw_votes', 'dem_nominee': 'gov_dem_nominee', 'republican_raw_votes':'gov_republican_raw_votes',"rep_nominee":"gov_rep_nominee", 'raw_county_vote_totals':'gov_raw_county_vote_totals'}, inplace=True)
govdata

,election_year,fips,county_name,state,sfips,office,election_type,gov_seat_status,gov_democratic_raw_votes,gov_dem_nominee,gov_republican_raw_votes,gov_rep_nominee,gov_raw_county_vote_totals_two_party,gov_raw_county_vote_totals
0,1865,1001.0,AUTAUGA,AL,1,GOV,G,Democratic Open Seat Uncontested by Republicans,391.0,Michael J. Bulger,0,None,391.0,757.0
1,1868,1001.0,AUTAUGA,AL,1,GOV,G,Third-Party/Independent Open Seat Uncontested ...,0.0,None,1754,William Hugh Smith,1754.0,1754.0
2,1870,1001.0,AUTAUGA,AL,1,GOV,G,Republican Open Seat,0.0,Robert B. Lindsay,0,William Hugh Smith,0.0,2317.0
3,1872,1001.0,AUTAUGA,AL,1,GOV,G,Democratic Open Seat Uncontested by Democrats,0.0,None,0,David P. Lewis,0.0,2274.0
4,1874,1001.0,AUTAUGA,AL,1,GOV,G,Republican Open Seat,778.0,George S. Houston,1648,David P. Lewis,2426.0,2426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149319,1910,NaN,JAMES,TN,21,GOV,G,Democratic Open Seat,174.0,Robert L. Taylor,521,Ben W. Hooper,695.0,695.0
149320,1912,NaN,JAMES,TN,21,GOV,G,Republican Open Seat,196.0,Benton McMillin,516,Ben W. Hooper,712.0,774.0
149321,1914,NaN,JAMES,TN,21,GOV,G,Republican Open Seat,156.0,Tom C. Rye,319,Ben W. Hooper,475.0,475.0
149322,1916,NaN,JAMES,TN,21,GOV,G,Democratic Open Seat,229.0,Tom C. Rye,606,John W. Overall,835.0,836.0


In [8]:
presdata.columns
presdata = presdata.drop(["county_first_date","county_end_date","state_admission_date","original_county_name","original_name_end_date","complete_county_cases"], axis=1)
presdata.rename(columns={'seat_status': 'pres_seat_status', 'democratic_raw_votes': 'pres_democratic_raw_votes', 'dem_nominee': 'pres_dem_nominee', 'republican_raw_votes':'pres_republican_raw_votes','republican_raw_votes':'pres_republican_raw_votes', 'raw_county_vote_totals':'pres_raw_county_vote_totals'}, inplace=True)
presdata

,election_year,fips,county_name,state,sfips,office,election_type,pres_seat_status,pres_democratic_raw_votes,pres_dem_nominee,pres_republican_raw_votes,rep_nominee,pres_raw_county_vote_totals_two_party,pres_raw_county_vote_totals
0,1868,1001.0,AUTAUGA,AL,1,PRES,G,Open Seat,851.0,Horatio Seymour,1505.0,Ulysses S. Grant,2356.0,2356.0
1,1872,1001.0,AUTAUGA,AL,1,PRES,G,Republican President Re-election,669.0,Horace Greeley,1593.0,Ulysses S. Grant,2262.0,2262.0
2,1876,1001.0,AUTAUGA,AL,1,PRES,G,Open Seat,804.0,Samuel J. Tilden,1576.0,Rutherford B. Hayes,2380.0,2380.0
3,1880,1001.0,AUTAUGA,AL,1,PRES,G,Open Seat,978.0,Winfield Scott Hancock,974.0,James A. Garfield,1952.0,1967.0
4,1884,1001.0,AUTAUGA,AL,1,PRES,G,Open Seat,911.0,Grover Cleveland,877.0,James G. Blaine,1788.0,1789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113751,1904,NaN,JAMES,TN,48,PRES,G,Elevated Republican President Election,222.0,Alton B. Parker,533.0,Theodore Roosevelt,755.0,763.0
113752,1908,NaN,MANCHESTER,VA,20,PRES,G,Open Seat,363.0,William Jennings Bryan,114.0,William Howard Taft,477.0,478.0
113753,1908,NaN,JAMES,TN,20,PRES,G,Open Seat,217.0,William Jennings Bryan,608.0,William Howard Taft,825.0,826.0
113754,1912,NaN,JAMES,TN,26,PRES,G,Republican President Re-election,202.0,Woodrow Wilson,169.0,William Howard Taft,371.0,779.0


In [9]:
fulldf = govdata.merge(sendata,"outer", on=['election_year', 'fips', 'county_name', 'state', 'sfips']).merge(presdata,"outer", on=['election_year', 'fips', 'county_name', 'state', 'sfips'])

In [10]:
fulldf_modern = fulldf[fulldf['election_year']>=1959]

In [11]:
fulldf_modern["fips"] = fulldf_modern["fips"].astype(int)
fulldf_modern

C:\Users\toomuchdust\AppData\Local\Temp\ipykernel_11684\1723311352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fulldf_modern["fips"] = fulldf_modern["fips"].astype(int)


,election_year,fips,county_name,state,sfips,office_x,election_type_x,gov_seat_status,gov_democratic_raw_votes,gov_dem_nominee,...,sen_raw_county_vote_totals,office,election_type,pres_seat_status,pres_democratic_raw_votes,pres_dem_nominee,pres_republican_raw_votes,rep_nominee,pres_raw_county_vote_totals_two_party,pres_raw_county_vote_totals
33,1962,1001,AUTAUGA,AL,1,GOV,G,Democratic Open Seat Uncontested by Republicans,1594.0,George C. Wallace,...,1849.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,1966,1001,AUTAUGA,AL,1,GOV,G,Democratic Open Seat,4664.0,Lurleen B. Wallace,...,6265.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1970,1001,AUTAUGA,AL,1,GOV,G,Democratic Open Seat Uncontested by Republicans,5206.0,George C. Wallace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,1974,1001,AUTAUGA,AL,1,GOV,G,Democratic Incumbent,3868.0,George C. Wallace,...,4181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,1978,1001,AUTAUGA,AL,1,GOV,G,Democratic Open Seat,4948.0,Forrest H. Jr. James,...,5939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231456,1968,56045,WESTON,WY,56,NaN,NaN,NaN,NaN,NaN,...,NaN,PRES,G,Open Seat,610.0,Herbert H. Humphrey,1497.0,Richard M. Nixon,2107.0,2373.0
231457,1980,56045,WESTON,WY,56,NaN,NaN,NaN,NaN,NaN,...,NaN,PRES,G,Democratic President Re-election,584.0,Jimmy Carter,2219.0,Ronald Reagan,2803.0,2970.0
231458,1992,56045,WESTON,WY,56,NaN,NaN,NaN,NaN,NaN,...,NaN,PRES,G,Republican President Re-election,727.0,Bill Clinton,1465.0,George H.W. Bush,2192.0,3027.0
231459,2004,56045,WESTON,WY,56,NaN,NaN,NaN,NaN,NaN,...,NaN,PRES,G,Republican President Re-election,580.0,John F. Kerry,2739.0,George W. Bush,3319.0,3392.0


In [13]:
fulldf_modern.to_csv('data/countyelectiondata.csv', index=False)